
## Group-Aware alpha-Fair Allocation: closed form, gradients, tests


In [44]:
# New toy example for Fair Decision Feedback Loop (FDFL)
# Focus: Prediction fairness with accuracy disparity

import numpy as np
import cvxpy as cp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

# Data Generation
def generate_data():
    """Generate feature and benefit data for two groups and two resources."""
    f1 = np.array([5, 10, 14, 17])  # Group 1 features
    f2 = np.array([4, 6, 9, 12])    # Group 2 features
    n = len(f1)                     # Number of individuals per group

    # Benefits for each group and resource
    a = 1/4 * f1 + 1  # Group 1, resource 1
    c = 1/3 * f1 + 2  # Group 1, resource 2
    b = 1/2 * f2 + 2  # Group 2, resource 1
    d = 3/16 * f2 + 3 # Group 2, resource 2

    # Print benefits and their means
    print("Group 1, resource 1:", a, f"Mean: {np.mean(a):.2f}")
    print("Group 1, resource 2:", c, f"Mean: {np.mean(c):.2f}")
    print("Group 2, resource 1:", b, f"Mean: {np.mean(b):.2f}")
    print("Group 2, resource 2:", d, f"Mean: {np.mean(d):.2f}")

    X = np.concatenate([f1, f2]).reshape(-1, 1)
    y1 = np.concatenate([a, b])  # Outcome for resource 1
    y2 = np.concatenate([c, d])  # Outcome for resource 2
    return f1, f2, a, b, c, d, X, y1, y2, n

# Standard MSE Regression
def standard_regression(X, y1, y2, a, b, c, d, n):
    """Perform standard linear regression for both resources."""
    # Resource 1
    reg1 = LinearRegression().fit(X, y1)
    pred1 = reg1.predict(X)
    mse1 = mean_squared_error(y1, pred1)
    ahat_mse = pred1[:n]
    bhat_mse = pred1[n:]

    # Resource 2
    reg2 = LinearRegression().fit(X, y2)
    pred2 = reg2.predict(X)
    mse2 = mean_squared_error(y2, pred2)
    chat_mse = pred2[:n]
    dhat_mse = pred2[n:]

    # Print results
    print("\nStandard MSE - Resource 1:", mse1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_mse - a), np.linalg.norm(bhat_mse - b))
    print("Standard MSE - Resource 2:", mse2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_mse - c), np.linalg.norm(dhat_mse - d))

    return ahat_mse, bhat_mse, chat_mse, dhat_mse

# Fair Regression with Accuracy Disparity
def fair_regression(X, y1, y2, f1, a, f2, b, c, d, n, lam=1):
    """Train fair regression model minimizing accuracy disparity."""
    initial_guess = [1.0, 1.0]
    bounds = [(-10, 10), (-10, 10)]

    def y_mse_loss_fair(params, x, y, x1, y1, x2, y2, lam):
        slope, intercept = params
        yhat = slope * x + intercept
        yg1 = yhat[:len(x1)]
        yg2 = yhat[len(x1):]
        disparity = np.abs(np.linalg.norm(yg1 - y1) - np.linalg.norm(yg2 - y2))
        mse = np.mean((y - yhat) ** 2)
        return mse + lam * disparity

    def train_fair_regression_scipy(x, y, x1, y1, x2, y2, lam):
        result = minimize(y_mse_loss_fair, initial_guess, args=(x, y, x1, y1, x2, y2, lam),
                         bounds=bounds, method='L-BFGS-B')
        return result.x, result.fun

    # Train for Resource 1
    w1, obj_val1 = train_fair_regression_scipy(X.flatten(), y1, f1, a, f2, b, lam)
    fpred1 = w1[0] * X.flatten() + w1[1]
    fmse1 = mean_squared_error(y1, fpred1)
    ahat_fair = fpred1[:n]
    bhat_fair = fpred1[n:]

    # Train for Resource 2
    w2, obj_val2 = train_fair_regression_scipy(X.flatten(), y2, f1, c, f2, d, lam)
    fpred2 = w2[0] * X.flatten() + w2[1]
    fmse2 = mean_squared_error(y2, fpred2)
    chat_fair = fpred2[:n]
    dhat_fair = fpred2[n:]

    # Print results
    print("\nMSE when fair - Resource 1:", fmse1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_fair - a), np.linalg.norm(bhat_fair - b))
    print("MSE when fair - Resource 2:", fmse2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_fair - c), np.linalg.norm(dhat_fair - d))

    return ahat_fair, bhat_fair, chat_fair, dhat_fair

# Optimal Decision Making
def opt_dec(a, b, c, d, n):
    """Compute optimal decisions to minimize utility difference."""
    # Ensure inputs are 1D arrays
    a, b, c, d = np.asarray(a).flatten(), np.asarray(b).flatten(), np.asarray(c).flatten(), np.asarray(d).flatten()
    
    x = cp.Variable(n, boolean=True)  # Group 1, resource 1
    y = cp.Variable(n, boolean=True)  # Group 1, resource 2
    s = cp.Variable(n, boolean=True)  # Group 2, resource 1
    t = cp.Variable(n, boolean=True)  # Group 2, resource 2

    # Total benefit for each group using CVXPY's element-wise multiplication
    u_G1 = cp.sum(cp.multiply(a, x) + cp.multiply(c, y))
    u_G2 = cp.sum(cp.multiply(b, s) + cp.multiply(d, t))

    # Minimize utility difference
    objective = cp.Minimize((u_G1 - u_G2) ** 2)
    constraints = [
        *[x[i] + y[i] == 1 for i in range(n)],
        *[s[i] + t[i] == 1 for i in range(n)],
        cp.sum(x) + cp.sum(s) == n,
        cp.sum(y) + cp.sum(t) == n
    ]

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS_BB)
    
    if prob.status != 'optimal':
        raise ValueError(f"Optimization failed with status: {prob.status}")
    
    return x.value.round(), y.value.round(), s.value.round(), t.value.round()

# End-to-End (E2E) Training
def e2e_training(X, a, b, c, d, n):
    """Train end-to-end model minimizing decision error."""
    initial_guess = [1, 1, 1, 1]
    bounds = [(0, 10), (0, 10), (0, 10), (0, 10)]

    def decision_loss(params, X, a, b, c, d):
        slope_r1, intercept_r1, slope_r2, intercept_r2 = params
        yhat_r1 = slope_r1 * X.flatten() + intercept_r1
        yhat_r2 = slope_r2 * X.flatten() + intercept_r2
        ap, bp, cp, dp = yhat_r1[:n], yhat_r1[n:], yhat_r2[:n], yhat_r2[n:]
        x, y, s, t = opt_dec(a, b, c, d, n)
        xp, yp, sp, tp = opt_dec(ap, bp, cp, dp, n)
        return np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp)

    result = minimize(decision_loss, initial_guess, args=(X, a, b, c, d), bounds=bounds, method='L-BFGS-B')
    k1_e2e, b1_e2e, k2_e2e, b2_e2e = result.x

    y1_e2e = k1_e2e * X.flatten() + b1_e2e
    e2e_r1 = mean_squared_error(y1, y1_e2e)
    ahat_e2e = y1_e2e[:n]
    bhat_e2e = y1_e2e[n:]

    y2_e2e = k2_e2e * X.flatten() + b2_e2e
    e2e_r2 = mean_squared_error(y2, y2_e2e)
    chat_e2e = y2_e2e[:n]
    dhat_e2e = y2_e2e[n:]

    print("\nE2E - Resource 1:", e2e_r1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_e2e - a), np.linalg.norm(bhat_e2e - b))
    print("E2E - Resource 2:", e2e_r2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_e2e - c), np.linalg.norm(dhat_e2e - d))

    return ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e

# Fair End-to-End Training
def fair_e2e_training(X, a, b, c, d, n, lam=1):
    """Train fair end-to-end model with accuracy disparity penalty."""
    initial_guess = [1, 1, 1, 1]
    bounds = [(0, 10), (0, 10), (0, 10), (0, 10)]

    def fair_decision_loss(params, X, a, b, c, d, lam):
        slope_r1, intercept_r1, slope_r2, intercept_r2 = params
        yhat_r1 = slope_r1 * X.flatten() + intercept_r1
        yhat_r2 = slope_r2 * X.flatten() + intercept_r2
        ap, bp, cp, dp = yhat_r1[:n], yhat_r1[n:], yhat_r2[:n], yhat_r2[n:]
        x, y, s, t = opt_dec(a, b, c, d, n)
        xp, yp, sp, tp = opt_dec(ap, bp, cp, dp, n)
        d_mse = np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp)
        acc_disparity = np.abs(np.linalg.norm(a - ap) - np.linalg.norm(b - bp)) + \
                        np.abs(np.linalg.norm(c - cp) - np.linalg.norm(d - dp))
        return d_mse + lam * acc_disparity

    result = minimize(fair_decision_loss, initial_guess, args=(X, a, b, c, d, lam), bounds=bounds, method='L-BFGS-B')
    k1_e2ef, b1_e2ef, k2_e2ef, b2_e2ef = result.x

    y1_e2ef = k1_e2ef * X.flatten() + b1_e2ef
    e2ef_r1 = mean_squared_error(y1, y1_e2ef)
    ahat_e2ef = y1_e2ef[:n]
    bhat_e2ef = y1_e2ef[n:]

    y2_e2ef = k2_e2ef * X.flatten() + b2_e2ef
    e2ef_r2 = mean_squared_error(y2, y2_e2ef)
    chat_e2ef = y2_e2ef[:n]
    dhat_e2ef = y2_e2ef[n:]

    print("\nFair E2E - Resource 1:", e2ef_r1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_e2ef - a), np.linalg.norm(bhat_e2ef - b))
    print("Fair E2E - Resource 2:", e2ef_r2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_e2ef - c), np.linalg.norm(dhat_e2ef - d))

    return ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef

# Utility and Decision Error Calculation
def print_decision_results(a, b, c, d, n, ahat, bhat, chat, dhat, label):
    """Calculate and print utility and decision error for given predictions."""
    x, y, s, t = opt_dec(a, b, c, d, n)
    xp, yp, sp, tp = opt_dec(ahat, bhat, chat, dhat, n)
    print(f"\n{label} predictions")
    print(f"Group 1, predicted utility: {np.dot(a, xp) + np.dot(c, yp):.2f}")
    print(f"Group 2, predicted utility: {np.dot(b, sp) + np.dot(d, tp):.2f}")
    print(f"Decision error: {np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp):.2f}")

# Main Execution
def main():
    # Generate data
    f1, f2, a, b, c, d, X, y1, y2, n = generate_data()

    # True utilities
    print("\nTrue values")
    x, y, s, t = opt_dec(a, b, c, d, n)
    print(f"Group 1, true utility: {np.dot(a, x) + np.dot(c, y):.2f}")
    print(f"Group 2, true utility: {np.dot(b, s) + np.dot(d, t):.2f}")

    # Standard MSE regression
    ahat_mse, bhat_mse, chat_mse, dhat_mse = standard_regression(X, y1, y2, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_mse, bhat_mse, chat_mse, dhat_mse, "MSE")

    # Fair regression
    ahat_fair, bhat_fair, chat_fair, dhat_fair = fair_regression(X, y1, y2, f1, a, f2, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_fair, bhat_fair, chat_fair, dhat_fair, "Fair")

    # End-to-end training
    ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e = e2e_training(X, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e, "E2E")

    # Fair end-to-end training
    ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef = fair_e2e_training(X, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef, "Fair E2E")

if __name__ == "__main__":
    main()

Group 1, resource 1: [2.25 3.5  4.5  5.25] Mean: 3.88
Group 1, resource 2: [3.66666667 5.33333333 6.66666667 7.66666667] Mean: 5.83
Group 2, resource 1: [4.  5.  6.5 8. ] Mean: 5.88
Group 2, resource 2: [3.75   4.125  4.6875 5.25  ] Mean: 4.45

True values
Group 1, true utility: 19.75
Group 2, true utility: 19.88

Standard MSE - Resource 1: 2.302980398457583
MSE by group - G1, G2: 2.7250826038412868 3.3162882851016224
Standard MSE - Resource 2: 0.08547529946800914
MSE by group - G1, G2: 0.36346671877677333 0.74276129414888

MSE predictions
Group 1, predicted utility: 16.92
Group 2, predicted utility: 18.06
Decision error: 5.46

MSE when fair - Resource 1: 2.3931863312455346
MSE by group - G1, G2: 3.093985304909456 3.093985388292682
MSE when fair - Resource 2: 0.955360648421686
MSE by group - G1, G2: 1.9548510409249535 1.954851041682745

Fair predictions
Group 1, predicted utility: 18.75
Group 2, predicted utility: 18.94
Decision error: 5.66

E2E - Resource 1: 48.17213559130881
MSE by g

In [52]:
# New toy example for Fair Decision Feedback Loop (FDFL)
# Focus: Prediction fairness with accuracy disparity and nonlinear benefits

import numpy as np
import cvxpy as cp
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

# Data Generation with Nonlinear Benefits
def generate_data():
    """Generate feature and benefit data for two groups and two resources with nonlinear benefits."""
    f1 = np.array([5, 10, 14, 17])  # Group 1 features
    f2 = np.array([4, 6, 9, 12])    # Group 2 features
    n = len(f1)                     # Number of individuals per group

    # Nonlinear benefits for each group and resource
    a = 0.5 * f1**0.9 + np.log(f1 + 1)  # Group 1, resource 1
    c = 0.3 * f1**1.1 + 0.5 * np.log(f1 + 1)  # Group 1, resource 2
    b = 0.4 * f2**0.95 + np.log(f2 + 1)  # Group 2, resource 1
    d = 0.2 * f2**1.05 + 0.7 * np.log(f2 + 1)  # Group 2, resource 2

    # Print benefits and their means
    print("Group 1, resource 1:", a, f"Mean: {np.mean(a):.2f}")
    print("Group 1, resource 2:", c, f"Mean: {np.mean(c):.2f}")
    print("Group 2, resource 1:", b, f"Mean: {np.mean(b):.2f}")
    print("Group 2, resource 2:", d, f"Mean: {np.mean(d):.2f}")

    X = np.concatenate([f1, f2]).reshape(-1, 1)
    y1 = np.concatenate([a, b])  # Outcome for resource 1
    y2 = np.concatenate([c, d])  # Outcome for resource 2
    return f1, f2, a, b, c, d, X, y1, y2, n

# Polynomial Regression (Standard MSE)
def standard_regression(X, y1, y2, a, b, c, d, n, degree=2):
    """Perform polynomial regression for both resources."""
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    # Resource 1
    reg1 = LinearRegression().fit(X_poly, y1)
    pred1 = reg1.predict(X_poly)
    mse1 = mean_squared_error(y1, pred1)
    ahat_mse = pred1[:n]
    bhat_mse = pred1[n:]

    # Resource 2
    reg2 = LinearRegression().fit(X_poly, y2)
    pred2 = reg2.predict(X_poly)
    mse2 = mean_squared_error(y2, pred2)
    chat_mse = pred2[:n]
    dhat_mse = pred2[n:]

    # Print results
    print("\nStandard MSE - Resource 1:", mse1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_mse - a), np.linalg.norm(bhat_mse - b))
    print("Standard MSE - Resource 2:", mse2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_mse - c), np.linalg.norm(dhat_mse - d))

    return ahat_mse, bhat_mse, chat_mse, dhat_mse

# Fair Polynomial Regression with Accuracy Disparity
def fair_regression(X, y1, y2, f1, a, f2, b, c, d, n, lam=1, degree=2):
    """Train fair polynomial regression model minimizing accuracy disparity."""
    initial_guess = [1.0] * (degree + 1)  # Coefficients for polynomial of given degree
    bounds = [(-10, 10)] * (degree + 1)

    def y_mse_loss_fair(params, x, y, x1, y1, x2, y2, lam):
        # Construct polynomial features manually
        yhat = np.zeros_like(x)
        for i in range(len(params)):
            yhat += params[i] * (x ** i)
        yg1 = yhat[:len(x1)]
        yg2 = yhat[len(x1):]
        disparity = np.abs(np.linalg.norm(yg1 - y1) - np.linalg.norm(yg2 - y2))
        mse = np.mean((y - yhat) ** 2)
        return mse + lam * disparity

    def train_fair_regression_scipy(x, y, x1, y1, x2, y2, lam):
        result = minimize(y_mse_loss_fair, initial_guess, args=(x, y, x1, y1, x2, y2, lam),
                         bounds=bounds, method='L-BFGS-B')
        return result.x, result.fun

    # Train for Resource 1
    w1, obj_val1 = train_fair_regression_scipy(X.flatten(), y1, f1, a, f2, b, lam)
    fpred1 = np.sum([w1[i] * (X.flatten() ** i) for i in range(len(w1))], axis=0)
    fmse1 = mean_squared_error(y1, fpred1)
    ahat_fair = fpred1[:n]
    bhat_fair = fpred1[n:]

    # Train for Resource 2
    w2, obj_val2 = train_fair_regression_scipy(X.flatten(), y2, f1, c, f2, d, lam)
    fpred2 = np.sum([w2[i] * (X.flatten() ** i) for i in range(len(w2))], axis=0)
    fmse2 = mean_squared_error(y2, fpred2)
    chat_fair = fpred2[:n]
    dhat_fair = fpred2[n:]

    # Print results
    print("\nMSE when fair - Resource 1:", fmse1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_fair - a), np.linalg.norm(bhat_fair - b))
    print("MSE when fair - Resource 2:", fmse2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_fair - c), np.linalg.norm(dhat_fair - d))

    return ahat_fair, bhat_fair, chat_fair, dhat_fair

# Optimal Decision Making
def opt_dec(a, b, c, d, n):
    """Compute optimal decisions to minimize utility difference."""
    a, b, c, d = np.asarray(a).flatten(), np.asarray(b).flatten(), np.asarray(c).flatten(), np.asarray(d).flatten()
    
    x = cp.Variable(n, boolean=True)  # Group 1, resource 1
    y = cp.Variable(n, boolean=True)  # Group 1, resource 2
    s = cp.Variable(n, boolean=True)  # Group 2, resource 1
    t = cp.Variable(n, boolean=True)  # Group 2, resource 2

    # Total benefit for each group
    u_G1 = cp.sum(cp.multiply(a, x) + cp.multiply(c, y))
    u_G2 = cp.sum(cp.multiply(b, s) + cp.multiply(d, t))

    # Minimize utility difference
    objective = cp.Minimize((u_G1 - u_G2) ** 2)
    constraints = [
        *[x[i] + y[i] == 1 for i in range(n)],
        *[s[i] + t[i] == 1 for i in range(n)],
        cp.sum(x) + cp.sum(s) == n,
        cp.sum(y) + cp.sum(t) == n
    ]

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS_BB)
    
    if prob.status != 'optimal':
        raise ValueError(f"Optimization failed with status: {prob.status}")
    
    return x.value.round(), y.value.round(), s.value.round(), t.value.round()

# End-to-End (E2E) Training
def e2e_training(X, a, b, c, d, n, degree=2):
    """Train end-to-end polynomial model minimizing decision error."""
    initial_guess = [1] * (degree + 1) * 2  # Coefficients for two polynomials
    bounds = [(0, 10)] * (degree + 1) * 2

    def decision_loss(params, X, a, b, c, d):
        half = len(params) // 2
        w1, w2 = params[:half], params[half:]
        yhat_r1 = np.sum([w1[i] * (X.flatten() ** i) for i in range(len(w1))], axis=0)
        yhat_r2 = np.sum([w2[i] * (X.flatten() ** i) for i in range(len(w2))], axis=0)
        ap, bp, cp, dp = yhat_r1[:n], yhat_r1[n:], yhat_r2[:n], yhat_r2[n:]
        x, y, s, t = opt_dec(a, b, c, d, n)
        xp, yp, sp, tp = opt_dec(ap, bp, cp, dp, n)
        return np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp)

    result = minimize(decision_loss, initial_guess, args=(X, a, b, c, d), bounds=bounds, method='L-BFGS-B')
    half = len(result.x) // 2
    w1, w2 = result.x[:half], result.x[half:]

    y1_e2e = np.sum([w1[i] * (X.flatten() ** i) for i in range(len(w1))], axis=0)
    e2e_r1 = mean_squared_error(y1, y1_e2e)
    ahat_e2e = y1_e2e[:n]
    bhat_e2e = y1_e2e[n:]

    y2_e2e = np.sum([w2[i] * (X.flatten() ** i) for i in range(len(w2))], axis=0)
    e2e_r2 = mean_squared_error(y2, y2_e2e)
    chat_e2e = y2_e2e[:n]
    dhat_e2e = y2_e2e[n:]

    print("\nE2E - Resource 1:", e2e_r1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_e2e - a), np.linalg.norm(bhat_e2e - b))
    print("E2E - Resource 2:", e2e_r2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_e2e - c), np.linalg.norm(dhat_e2e - d))

    return ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e

# Fair End-to-End Training
def fair_e2e_training(X, a, b, c, d, n, lam=1, degree=2):
    """Train fair end-to-end polynomial model with accuracy disparity penalty."""
    initial_guess = [1] * (degree + 1) * 2
    bounds = [(0, 10)] * (degree + 1) * 2

    def fair_decision_loss(params, X, a, b, c, d, lam):
        half = len(params) // 2
        w1, w2 = params[:half], params[half:]
        yhat_r1 = np.sum([w1[i] * (X.flatten() ** i) for i in range(len(w1))], axis=0)
        yhat_r2 = np.sum([w2[i] * (X.flatten() ** i) for i in range(len(w2))], axis=0)
        ap, bp, cp, dp = yhat_r1[:n], yhat_r1[n:], yhat_r2[:n], yhat_r2[n:]
        x, y, s, t = opt_dec(a, b, c, d, n)
        xp, yp, sp, tp = opt_dec(ap, bp, cp, dp, n)
        d_mse = np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp)
        acc_disparity = np.abs(np.linalg.norm(a - ap) - np.linalg.norm(b - bp)) + \
                        np.abs(np.linalg.norm(c - cp) - np.linalg.norm(d - dp))
        return d_mse + lam * acc_disparity

    result = minimize(fair_decision_loss, initial_guess, args=(X, a, b, c, d, lam), bounds=bounds, method='L-BFGS-B')
    half = len(result.x) // 2
    w1, w2 = result.x[:half], result.x[half:]

    y1_e2ef = np.sum([w1[i] * (X.flatten() ** i) for i in range(len(w1))], axis=0)
    e2ef_r1 = mean_squared_error(y1, y1_e2ef)
    ahat_e2ef = y1_e2ef[:n]
    bhat_e2ef = y1_e2ef[n:]

    y2_e2ef = np.sum([w2[i] * (X.flatten() ** i) for i in range(len(w2))], axis=0)
    e2ef_r2 = mean_squared_error(y2, y2_e2ef)
    chat_e2ef = y2_e2ef[:n]
    dhat_e2ef = y2_e2ef[n:]

    print("\nFair E2E - Resource 1:", e2ef_r1)
    print("MSE by group - G1, G2:", np.linalg.norm(ahat_e2ef - a), np.linalg.norm(bhat_e2ef - b))
    print("Fair E2E - Resource 2:", e2ef_r2)
    print("MSE by group - G1, G2:", np.linalg.norm(chat_e2ef - c), np.linalg.norm(dhat_e2ef - d))

    return ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef

# Utility and Decision Error Calculation
def print_decision_results(a, b, c, d, n, ahat, bhat, chat, dhat, label):
    """Calculate and print utility and decision error for given predictions."""
    x, y, s, t = opt_dec(a, b, c, d, n)
    xp, yp, sp, tp = opt_dec(ahat, bhat, chat, dhat, n)
    print(f"\n{label} predictions")
    print(f"Group 1, predicted utility: {np.dot(a, xp) + np.dot(c, yp):.2f}")
    print(f"Group 2, predicted utility: {np.dot(b, sp) + np.dot(d, tp):.2f}")
    print(f"Decision error: {np.linalg.norm(x - xp) + np.linalg.norm(y - yp) + np.linalg.norm(s - sp) + np.linalg.norm(t - tp):.2f}")

# Main Execution
def main():
    # Generate data
    f1, f2, a, b, c, d, X, y1, y2, n = generate_data()

    # True utilities
    print("\nTrue values")
    x, y, s, t = opt_dec(a, b, c, d, n)
    print(f"Group 1, true utility: {np.dot(a, x) + np.dot(c, y):.2f}")
    print(f"Group 2, true utility: {np.dot(b, s) + np.dot(d, t):.2f}")

    # Standard MSE regression (polynomial)
    ahat_mse, bhat_mse, chat_mse, dhat_mse = standard_regression(X, y1, y2, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_mse, bhat_mse, chat_mse, dhat_mse, "MSE")

    # Fair polynomial regression
    ahat_fair, bhat_fair, chat_fair, dhat_fair = fair_regression(X, y1, y2, f1, a, f2, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_fair, bhat_fair, chat_fair, dhat_fair, "Fair")

    # End-to-end training
    ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e = e2e_training(X, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_e2e, bhat_e2e, chat_e2e, dhat_e2e, "E2E")

    # Fair end-to-end training
    ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef = fair_e2e_training(X, a, b, c, d, n)
    print_decision_results(a, b, c, d, n, ahat_e2ef, bhat_e2ef, chat_e2ef, dhat_e2ef, "Fair E2E")

if __name__ == "__main__":
    main()

Group 1, resource 1: [3.92010928 6.36953645 8.08437176 9.29323216] Mean: 6.92
Group 1, resource 2: [2.65780815 4.97572387 6.82244801 8.21559742] Mean: 5.67
Group 2, resource 1: [3.1022907  4.14024893 5.52803555 6.80413246] Mean: 4.89
Group 2, resource 2: [1.98402531 2.67460536 3.62083128 4.51296925] Mean: 3.20

True values
Group 1, true utility: 22.67
Group 2, true utility: 19.57

Standard MSE - Resource 1: 0.02653498466702308
MSE by group - G1, G2: 0.34091546499942527 0.30992986797082056
Standard MSE - Resource 2: 0.17225324675677112
MSE by group - G1, G2: 0.8185897791759223 0.8413897714394225

MSE predictions
Group 1, predicted utility: 22.67
Group 2, predicted utility: 19.57
Decision error: 0.00


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

In [50]:
"""
============================================================
VERIFY group-wise α-fair closed form against CVXPY
(one global budget, any #groups, continuous doses)
============================================================
pip install torch cvxpy numpy
"""

import numpy as np
import cvxpy as cp
import torch


# --------------------------------------------------------------------
# CLOSED-FORM ALLOCATION   (covers α = 0,1,∞ + finite α)
# --------------------------------------------------------------------
def closed_form_group_alpha(
    b_hat: np.ndarray,
    cost: np.ndarray,
    group: np.ndarray,
    Q: float,
    alpha,
):
    """
    Parameters
    ----------
    b_hat : (N,T)  positive utilities
    cost  : (N,T)  positive costs
    group : (N,)   integers 0..K-1
    Q     : float  budget
    alpha : float >0 or 0 or 1 or np.inf
    Returns
    -------
    d_star : (N,T) optimal doses
    """
    N, T = b_hat.shape
    K = group.max() + 1

    # per-group best benefit-cost ratio ρ_k and its argmax (i*,t*)
    rho_k = np.zeros(K)
    idx_k = np.zeros((K, 2), dtype=int)
    for k in range(K):
        mask = group == k
        ratio = (b_hat[mask] / cost[mask]).reshape(-1)
        i_star = ratio.argmax()
        i_glob = np.where(mask)[0][i_star // T]
        t_glob = i_star % T
        rho_k[k] = ratio.max()
        idx_k[k] = [i_glob, t_glob]

    p_k = rho_k / np.bincount(group)  # p_k = ρ_k / G_k

    # ------------ Stage I: allocate budget B_k = x_k ------------
    if alpha == 0:  # utilitarian
        winners = np.flatnonzero(p_k == p_k.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:  # logarithmic
        x = np.full(K, Q / K)
    elif alpha == np.inf:  # max-min
        inv = 1 / p_k
        x = Q * inv / inv.sum()
    else:  # generic  (0<α<∞, α≠1)
        weights = p_k ** (1 / alpha - 1)
        x = Q * weights / weights.sum()

    # ------------ Stage II: spend each x_k on its best item -----
    d_star = np.zeros_like(b_hat)
    for k in range(K):
        i, t = idx_k[k]
        d_star[i, t] = x[k] / cost[i, t]

    return d_star


# --------------------------------------------------------------------
# CVXPY benchmark (same problem)
# --------------------------------------------------------------------
def solve_cvxpy(b_hat, cost, group, Q, alpha):
    N, T = b_hat.shape
    d = cp.Variable((N, T), nonneg=True)

    util = []
    for k in range(group.max() + 1):
        idx = group == k
        util_k = cp.sum(cp.multiply(b_hat[idx], d[idx])) / idx.sum()
        util.append(util_k)
    util = cp.hstack(util)

    if alpha == 0:
        obj = cp.Maximize(cp.sum(util))
    elif alpha == 1:
        obj = cp.Maximize(cp.sum(cp.log(util)))
    elif alpha == np.inf:
        obj = cp.Maximize(cp.min(util))
    else:
        obj = cp.Maximize(cp.sum(util ** (1 - alpha) / (1 - alpha)))

    prob = cp.Problem(obj, [cp.sum(cp.multiply(cost, d)) <= Q])
    prob.solve()
    return d.value


# --------------------------------------------------------------------
# quick numerical experiment
# --------------------------------------------------------------------
def one_run(N=4, T=4, K=3, Q=200.0, alpha=2.0, seed=0):
    rng = np.random.default_rng(seed)
    b = 1 + 4 * rng.random((N, T))
    c = 0.2 + 0.8 * rng.random((N, T))
    g = rng.integers(0, K, size=N)

    d_cf = closed_form_group_alpha(b, c, g, Q, alpha)
    d_cvx = solve_cvxpy(b, c, g, Q, alpha)
    gap = np.abs(d_cf - d_cvx).max()
    print('d_cf:\n', d_cf)
    print('d_cvx:\n', d_cvx)

    print(f"α={alpha:5},  max|closed-form − solver| = {gap:.3e}")


if __name__ == "__main__":
    for a in [0, 1, 0.5, 2.0, 5.0, np.inf]:
        one_run(alpha=a)


d_cf:
 [[  0.           0.           0.           0.        ]
 [898.24768394   0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]]
d_cvx:
 [[  0.           0.           0.           0.        ]
 [898.24768394   0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]]
α=    0,  max|closed-form − solver| = 0.000e+00
d_cf:
 [[  0.           0.           0.           0.        ]
 [299.41589465   0.           0.           0.        ]
 [  0.         131.50747217   0.           0.        ]
 [ 89.07492632   0.           0.           0.        ]]
d_cvx:
 [[1.25020129e-05 1.34577150e-05 1.60728288e-05 1.15721217e-05]
 [2.99373760e+02 8.41978198e-05 6.81399539e-06 7.38655655e-06]
 [1.06880544e-05 1.31522426e+02 7.09596435e-06 5.08799434e-06]
 [8.90771415e+01 8.19658983e-06 4.33705917e-05 1.71885029e-05]]
α=    

In [51]:
"""
group_alpha_derivs.py
=====================

Closed form, objective gradient, and Jacobian for
  • single global budget Q
  • continuous doses
  • any α in {0, 1, ∞} ∪ (0, ∞)
  • arbitrary number of groups K (ties assumed measure-zero)

Run this file to see a finite-difference sanity check.
"""

import numpy as np


# ---------------------------------------------------------------------
# closed-form allocation  (two-stage: pick best item per group, then budget)
# ---------------------------------------------------------------------
def closed_form_group_alpha(b, c, g, Q, alpha):
    """
    Parameters
    ----------
    b : (N,T) positive predicted benefit   (numpy)
    c : (N,T) positive cost
    g : (N,)  integer group labels 0..K-1
    Q : float budget
    alpha : float >0   or 0   or 1   or np.inf
    Returns
    -------
    d_star : (N,T) allocation (numpy)
    idx_k  : (K,2) index (i,t) of best item for each group (needed later)
    """
    N, T = b.shape
    K = g.max() + 1

    # ----- best item (i_k,t_k) & ratio ρ_k in each group --------------
    rho = np.zeros(K)
    idx_k = np.zeros((K, 2), dtype=int)

    for k in range(K):
        mask = g == k
        ratio = (b[mask] / c[mask]).reshape(-1)
        flat_idx = ratio.argmax()
        i_global = np.where(mask)[0][flat_idx // T]
        t_global = flat_idx % T
        idx_k[k] = i_global, t_global
        rho[k] = ratio.max()

    G = np.bincount(g, minlength=K)
    p = rho / G                       # p_k = ρ_k / G_k

    # ---------- Stage I : budgets x_k = B_k ---------------------------
    if alpha == 0:                    # utilitarian
        winners = np.flatnonzero(p == p.max())
        x = np.zeros(K)
        x[winners] = Q / len(winners)
    elif alpha == 1:                  # log utility
        x = np.full(K, Q / K)
    elif alpha == np.inf:             # max–min
        inv = 1 / p
        x = Q * inv / inv.sum()
    else:                             # generic α
        beta = 1.0 / alpha
        weights = p ** (beta - 1)     # p^{1/α - 1}
        x = Q * weights / weights.sum()

    # ---------- Stage II : spend within group on best (i,t) -----------
    d_star = np.zeros_like(b)
    for k, (i, t) in enumerate(idx_k):
        d_star[i, t] = x[k] / c[i, t]

    return d_star, idx_k, p, x


# ---------------------------------------------------------------------
# gradient of objective  W(b)  w.r.t. each b_{it}
# ---------------------------------------------------------------------
def grad_W_wrt_b(b, c, g, Q, alpha):
    """
    Returns  ∇_b W  with the same shape as b
    """
    d_star, idx_k, p, _ = closed_form_group_alpha(b, c, g, Q, alpha)
    K = len(p)
    G = np.bincount(g, minlength=K)
    grad = np.zeros_like(b)

    if alpha in (0, np.inf):
        # objective is non-smooth here; return NaNs
        grad[:] = np.nan
        return grad

    if alpha == 1:                                  # W = Σ log u_k
        for k, (i, t) in enumerate(idx_k):
            grad[i, t] = 1 / (p[k] * G[k] * c[i, t])
        return grad

    # ---------- generic  0<α<∞, α≠1  -------------------------------
    beta = 1.0 / alpha
    D = np.sum(p ** (beta - 1))                     # denominator
    u = Q * p ** beta / D                          # group utilities

    # ∂W/∂u_k  and helper coeffs
    dW_du = u ** (-alpha)                          # u_k^{−α}
    coeff = Q ** (-alpha) * D ** (alpha - 2)       # common front factor

    for k, (i, t) in enumerate(idx_k):
        pk = p[k]
        term = (beta * D - (beta - 1) * pk ** (beta - 1))
        dW_dpk = coeff * pk ** (beta - 2) * term
        grad[i, t] = dW_dpk / (G[k] * c[i, t])     # dp/db = 1/(G_k c)
    return grad


# ---------------------------------------------------------------------
# full Jacobian  ∂d*/∂b   (sparse dictionary representation)
# ---------------------------------------------------------------------
def jacobian_d_wrt_b(b, c, g, Q, alpha):
    """
    Returns
    -------
    J : dict mapping (i*,t*)  ->  gradient row (N,T) as numpy array
        Only K rows are non-zero: one per group winner (i*,t*).
    """
    d_star, idx_k, p, x = closed_form_group_alpha(b, c, g, Q, alpha)
    K = len(p)
    G = np.bincount(g, minlength=K)
    N, T = b.shape
    J = {}

    # special / non-smooth cases --------------------------------------
    if alpha in (0, 1, np.inf):
        # Jacobian exists but is piecewise-constant & sparse:
        #  ∂d*(winner k)/∂b(winner k) via budget split; others zero.
        # Users typically rely on sub-gradients → return NaNs.
        for k, (i, t) in enumerate(idx_k):
            J[(i, t)] = np.full_like(b, np.nan)
        return J

    # ---------- generic  0<α<∞, α≠1 ----------------------------------
    beta = 1.0 / alpha
    D = np.sum(p ** (beta - 1))
    dD_dpk = (beta - 1) * p ** (beta - 2)          # derivative of D
    # pre-compute    ∂x_l / ∂p_k   for every pair (l,k)
    x_grad = np.zeros((K, K))
    for l in range(K):
        for k in range(K):
            if k == l:
                num = (beta - 1) * p[k] ** (beta - 2) * D
                num -= p[k] ** (beta - 1) * dD_dpk[k]
                x_grad[l, k] = Q * num / D ** 2
            else:
                x_grad[l, k] = -Q * p[l] ** (beta - 1) * dD_dpk[k] / D ** 2

    # build Jacobian rows (only one non-zero col per group)
    for k, (i_win, t_win) in enumerate(idx_k):
        row = np.zeros_like(b)
        # effect of p_k on every x_l  (thus on every winner l)
        for l, (i_l, t_l) in enumerate(idx_k):
            row[i_l, t_l] += (
                x_grad[l, k] / c[i_l, t_l] / (G[k] * c[i_win, t_win])
            )
        J[(i_win, t_win)] = row
    return J


# ---------------------------------------------------------------------
# quick finite-difference check  (generic α)
# ---------------------------------------------------------------------
def finite_difference_check():
    N, T, K = 10, 3, 3
    Q, alpha = 100.0, 2.0
    rng = np.random.default_rng(0)
    b = 1 + 4 * rng.random((N, T))
    c = 0.3 + 0.7 * rng.random((N, T))
    g = rng.integers(0, K, size=N)

    # gradient of W ---------------------------------------------------
    grad_analytic = grad_W_wrt_b(b, c, g, Q, alpha)
    eps = 1e-6
    grad_fd = np.zeros_like(b)
    for i in range(N):
        for t in range(T):
            b_plus = b.copy()
            b_minus = b.copy()
            b_plus[i, t] += eps
            b_minus[i, t] -= eps
            W_plus = objective_value(b_plus, c, g, Q, alpha)
            W_minus = objective_value(b_minus, c, g, Q, alpha)
            grad_fd[i, t] = (W_plus - W_minus) / (2 * eps)
    print("max|∇W_fd − ∇W_an| =", np.abs(grad_fd - grad_analytic).max())

    # Jacobian of d* --------------------------------------------------
    J = jacobian_d_wrt_b(b, c, g, Q, alpha)
    worst = 0.0
    for (i0, t0), row in J.items():
        b_eps = b.copy()
        b_eps[i0, t0] += eps
        d_plus, *_ = closed_form_group_alpha(b_eps, c, g, Q, alpha)
        d_orig, *_ = closed_form_group_alpha(b, c, g, Q, alpha)
        fd_col = (d_plus - d_orig) / eps
        worst = max(worst, np.abs(fd_col - row).max())
    print("max|Jac_fd − Jac_an| =", worst)


def objective_value(b, c, g, Q, alpha):
    d, *_ = closed_form_group_alpha(b, c, g, Q, alpha)
    K = g.max() + 1
    G = np.bincount(g, minlength=K)
    util = np.zeros(K)
    for k in range(K):
        i, t = np.where((g == k).reshape(-1, 1))
    # faster: read utilities from d:
    for k in range(K):
        mask = g == k
        util[k] = (b[mask] * d[mask]).sum() / G[k]

    if alpha == 0:
        return util.sum()
    if alpha == 1:
        return np.log(util).sum()
    if alpha == np.inf:
        return util.min()
    return np.sum(util ** (1 - alpha) / (1 - alpha))


# ---------------------------------------------------------------------
if __name__ == "__main__":
    finite_difference_check()


max|∇W_fd − ∇W_an| = 0.0029929353122838793
max|Jac_fd − Jac_an| = 1.4945091919571496e-06
